In [1]:
import keras
keras.__version__

2022-07-24 17:30:59.245540: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-24 17:30:59.245593: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


'2.8.0'

In [2]:
import numpy as np
import pandas as pd
from PIL import Image


f = pd.read_csv('./train_images.csv',header=1)
f = f.sample(frac=1).reset_index(drop=True)
x_lavel  = f.iloc[:,1:]
x_test = f.iloc[:,0]
_x_test = np.array(x_test)
_x_lavel = np.array(x_lavel)

train_image = []

for line in _x_test:
    line = line.rstrip()
    l = line.split()
    # データを読み込んで50x50に拡大
    img = Image.open(l[0])
    img = img.resize((50, 50))
    # 一列にした後、0-1のfloat値にする
    _img = np.asarray(img)
    
    train_image.append(_img.astype(np.float32)/255.0)
    #check_image.append(_img)
    # ラベルを1-of-k方式で用意する
    #train_label.append(tmp)
# numpy形式に変換
train_image = np.asarray(train_image)


train_image

array([[[[0.21568628, 0.03137255, 0.        ],
         [0.21568628, 0.03137255, 0.        ],
         [0.21176471, 0.03529412, 0.        ],
         ...,
         [0.21960784, 0.04705882, 0.        ],
         [0.21176471, 0.03921569, 0.        ],
         [0.1882353 , 0.01960784, 0.        ]],

        [[0.3137255 , 0.15294118, 0.04313726],
         [0.3137255 , 0.15294118, 0.04313726],
         [0.30980393, 0.15686275, 0.04313726],
         ...,
         [0.31764707, 0.16862746, 0.03921569],
         [0.30980393, 0.16078432, 0.03921569],
         [0.2901961 , 0.14117648, 0.03921569]],

        [[0.5921569 , 0.4745098 , 0.3254902 ],
         [0.5921569 , 0.4745098 , 0.3254902 ],
         [0.5882353 , 0.47843137, 0.3254902 ],
         ...,
         [0.59607846, 0.4862745 , 0.31764707],
         [0.5882353 , 0.48235294, 0.31764707],
         [0.5803922 , 0.47058824, 0.31764707]],

        ...,

        [[0.20784314, 0.7176471 , 0.4745098 ],
         [0.20784314, 0.7176471 , 0.4745098 ]

In [3]:
from keras.utils import np_utils

x_one_hot = np_utils.to_categorical(_x_lavel)
x_one_hot

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [4]:
from sklearn.model_selection import train_test_split


# 教師データとテストデータに分割
x_train, x_test, y_train, y_test= train_test_split(train_image, x_one_hot, train_size=0.8)

In [10]:
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(layers.Conv2D(32, (1, 1), activation='relu', input_shape=(50, 50, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (1, 1), activation='relu'))
model.add(layers.Conv2D(64, (1, 1), activation='relu'))
model.add(layers.Conv2D(64, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (1, 1), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(7744, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [11]:
model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=["accuracy"])

In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 50, 50, 32)        128       
                                                                 
 conv2d_13 (Conv2D)          (None, 48, 48, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 24, 24, 32)        1056      
                                                                 
 conv2d_15 (Conv2D)          (None, 24, 24, 64)        2112      
                                                                 
 conv2d_16 (Conv2D)          (None, 23, 23, 64)        16448     
                                                      

In [13]:
history = model.fit(x_train, y_train, epochs=50, batch_size=32, verbose=1, validation_split=0.1)

Epoch 1/50
18/18 [==============================] - 8s 400ms/step - loss: 0.6719 - accuracy: 0.6996 - val_loss: 0.6190 - val_accuracy: 0.7258
Epoch 2/50
18/18 [==============================] - 6s 361ms/step - loss: 0.6228 - accuracy: 0.7158 - val_loss: 0.6133 - val_accuracy: 0.7258
Epoch 3/50
18/18 [==============================] - 7s 402ms/step - loss: 0.6083 - accuracy: 0.7158 - val_loss: 0.5935 - val_accuracy: 0.7258
Epoch 4/50
18/18 [==============================] - 8s 428ms/step - loss: 0.6046 - accuracy: 0.7158 - val_loss: 0.5901 - val_accuracy: 0.7258
Epoch 5/50
18/18 [==============================] - 7s 389ms/step - loss: 0.5985 - accuracy: 0.7158 - val_loss: 0.5867 - val_accuracy: 0.7258
Epoch 6/50
18/18 [==============================] - 7s 365ms/step - loss: 0.6044 - accuracy: 0.7158 - val_loss: 0.6107 - val_accuracy: 0.7258
Epoch 7/50
18/18 [==============================] - 7s 361ms/step - loss: 0.6012 - accuracy: 0.7158 - val_loss: 0.5868 - val_accuracy: 0.7258
Epoch 

In [ ]:
plt.plot(history.epoch, history.history["accuracy"], label="acc")
plt.plot(history.epoch, history.history["loss"], label="loss")
plt.xlabel("epoch")
plt.legend()

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)

print("Test score", score[0])
print("Test accuracy", score[1])

In [ ]:
model.save('enter_model_mfcc.h5')

In [14]:
import tensorflow as tf
from tensorflow import keras

load_model = keras.models.load_model('enter_model_mfcc.h5')

load_model.evaluate(x_test, y_test, verbose=1)

5/5 [==============================] - 0s 43ms/step - loss: 0.1202 - accuracy: 0.9548


[0.12020426988601685, 0.9548386931419373]

In [ ]:
hoge = load_model.predict(x_test, batch_size=32, verbose=0, steps=None)

for i in range(160):
    if hoge[i][0] >= 0.6:
        print(hoge[i])

In [25]:
# ※※※　　　　　　　　　　　　　　　　　　　　　　　　　　※※※
# 推論試験用、学習データ作成、試験時はこのブロックは実行しないこと 
# ※※※　　　　　　　　　　　　　　　　　　　　　　　　　　※※※

img = Image.open('./capcha.jpg')
img_ = np.asarray(img.resize((50,50)))
print(img_.shape)
mfcc_ = np.asarray([img_.astype(np.float32)/255.0])
print(mfcc_.shape)
hoge2 = load_model.predict(mfcc_, batch_size=32, verbose=0, steps=None)

print(hoge2)

(50, 50, 3)
(1, 50, 50, 3)
[[0.02381883 0.9761812 ]]
